# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:

from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-139374
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-139374


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
ws = Workspace.from_config()
experiment_name = 'capstoneexperiment'

experiment=Experiment(ws, experiment_name)
ds = Dataset.get_by_name(ws,'alcohol')
ds1=ds.to_pandas_dataframe()

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification



x= pd.DataFrame(ds1)
y= ds1.drop("Walc", inplace=True, axis=1)

    

x_train, x_test = train_test_split(x, test_size=0.2)
y_train, y_test = train_test_split(y, test_size=0.2)

parser = argparse.ArgumentParser()

parser.add_argument('--n_estimators', type=int, default=100, help="Inverse of regularization strength. Smaller values cause stronger regularization")

args = parser.parse_args()

run.log("n_estimators:", np.int(args.n-estimators))
run.log("min_samlples_split", np.int(args.min_samples_split))

model = RandomForestClassifier(n_estimators = args.n_estimators, min_samples_split = args.min_samples_split)

accuracy = model.score(x_test, y_test)
run.log("Accuracy", np.float(accuracy))
os.makedirs('outputs',exist_ok=True)
joblib.dump(model, 'outputs/model.joblib')





TypeError: Expected sequence or array-like, got <class 'NoneType'>

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice,uniform
import os
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        '--n_estimators': choice(50,100,150),
        '--min_samples_split': choice(2,4,6)
    }
)
        


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
   source_directory='./',
   entry_script='train.py',
   compute_target=cpu_cluster_name,
   #use_docker=False,
   #script_params = {'--reg_rate': 0.07},
   #conda_packages=['scikit-learn']
   )

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                hyperparameter_sampling=ps,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=6,
                                max_concurrent_runs=4)
 #----------------------------------------------------------------------
#from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn
#from azureml.train.hyperdrive.run import PrimaryMetricGoal
#from azureml.train.hyperdrive.policy import BanditPolicy
#from azureml.train.hyperdrive.sampling import RandomParameterSampling
#from azureml.train.hyperdrive.runconfig import HyperDriveConfig
#from azureml.train.hyperdrive.parameter_expressions import choice,uniform
#import os

# Specify parameter sampler
#ps = RandomParameterSampling(
#    {
#        '--C': choice(0.001,0.01,0.1,1,10,100),
#        '--max_iter': choice(8,100,120)
#    }
#)

# Specify a Policy
#policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#if "training" not in os.listdir():
#    os.mkdir("./training")# Create a SKLearn estimator for use with train.py
#est = SKLearn(
#    source_directory='./',
#    entry_script='train.py',
#    compute_target=cpu_cluster_name,
    #use_docker=False,
    #script_params = {'--reg_rate': 0.07},
    #conda_packages=['scikit-learn']
#    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = HyperDriveConfig(estimator=est,
#                                hyperparameter_sampling=ps,
#                                policy=policy,
#                                primary_metric_name='Accuracy',
#                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
#                                max_total_runs=6,
#                               max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [9]:
#TODO: Submit your experiment
hdr = exp.submit(config = HyperDriveConfig)
RunDetails(hdr).show()

AzureMLException: AzureMLException:
	Message: Method to be submitted has not been registered
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Method to be submitted has not been registered"
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service